In [1]:
import pandas as pd
import tqdm
import sqlite3 as sq
import matplotlib.pyplot as plt

## Import UIC Data

In [16]:
path = "../../dataset/"
data_path = path+ "/yelp_dataset_UIC"

res_name = data_path + '/yelpResData.db'
hot_name = data_path + '/yelpHotelData.db'

res_conn = sq.connect(res_name)
hot_conn = sq.connect(hot_name)

In [3]:
res_tables = res_conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for i,name in enumerate(res_tables):
    print(name[0])
    
print("")

hot_tables = hot_conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in hot_tables:
    print(name[0])

review
restaurant
reviewer

review
sqlite_stat1
sqlite_stat2
reviewer
hotel


In [33]:
sql_q = lambda x: "SELECT * FROM " + x + ";"
res_conn.text_factory = lambda b: b.decode(errors = 'ignore')
hot_conn.text_factory = lambda b: b.decode(errors = 'ignore')

res_rev_df = pd.read_sql(sql_q('review'),res_conn)
res_rest_df = pd.read_sql(sql_q('restaurant'),res_conn)
res_rever_df = pd.read_sql(sql_q('reviewer'),res_conn)

hot_rev_df = pd.read_sql(sql_q('review'),hot_conn)
hot_sqlite1_df = pd.read_sql(sql_q('sqlite_stat1'),hot_conn)
hot_sqlite2_df = pd.read_sql(sql_q('sqlite_stat2'),hot_conn)
hot_rever_df = pd.read_sql(sql_q('reviewer'),hot_conn)
hot_hot_df = pd.read_sql(sql_q('hotel'),hot_conn)

## Table Columns

In [5]:
print("restaurant reviews: ",list(res_rev_df.columns.values))
print("restaurant restaurants: ",list(res_rest_df.columns.values))
print("restaurant reviewers: ",list(res_rever_df.columns.values))
print("")
print("hotel reviews: ",list(hot_rev_df.columns.values))
print("hotel sqlite_stat1: ",list(hot_sqlite1_df.columns.values))
print("hotel sqlite_stat2: ",list(hot_sqlite2_df.columns.values))
print("hotel reviewers: ",list(hot_rever_df.columns.values))
print("hotel hotel: ",list(hot_hot_df.columns.values))

restaurant reviews:  ['date', 'reviewID', 'reviewerID', 'reviewContent', 'rating', 'usefulCount', 'coolCount', 'funnyCount', 'flagged', 'restaurantID']
restaurant restaurants:  ['restaurantID', 'name', 'location', 'reviewCount', 'rating', 'categories', 'address', 'Hours', 'GoodforKids', 'AcceptsCreditCards', 'Parking', 'Attire', 'GoodforGroups', 'PriceRange', 'TakesReservations', 'Delivery', 'Takeout', 'WaiterService', 'OutdoorSeating', 'WiFi', 'GoodFor', 'Alcohol', 'NoiseLevel', 'Ambience', 'HasTV', 'Caters', 'WheelchairAccessible', 'webSite', 'phoneNumber', 'filReviewCount']
restaurant reviewers:  ['reviewerID', 'name', 'location', 'yelpJoinDate', 'friendCount', 'reviewCount', 'firstCount', 'usefulCount', 'coolCount', 'funnyCount', 'complimentCount', 'tipCount', 'fanCount']

hotel reviews:  ['date', 'reviewID', 'reviewerID', 'reviewContent', 'rating', 'usefulCount', 'coolCount', 'funnyCount', 'flagged', 'hotelID']
hotel sqlite_stat1:  ['tbl', 'idx', 'stat']
hotel sqlite_stat2:  ['tbl

In [6]:
# Find unique values of flagged
res_rev_df.loc[:,'flagged'].unique()

array(['N', 'Y', 'NR', 'YR'], dtype=object)

In [7]:
print(len(res_rev_df['reviewID'].unique()))

788471


In [8]:
concat = pd.concat([res_rev_df, hot_rev_df])
print('Total number is {}.'.format(concat.shape[0]))
print('Unique number is {}.'.format(len(concat['reviewID'].unique())))
print(concat.shape[0]-len(concat['reviewID'].unique()))

/Library/Python/3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


Total number is 1476800.
Unique number is 1295536.
181264


In [34]:
# First relabel restaurantID and hotelID to businessID
res_rev_df.rename({'restaurantID':'businessID'},axis=1, inplace=True)
hot_rev_df.rename({'hotelID':'businessID'},axis=1, inplace=True)

print(list(res_rev_df.columns.values))
print(list(hot_rev_df.columns.values))

# Remove duplicates
hotel_keep = ~hot_rev_df['reviewID'].isin(res_rev_df['reviewID'])

print(sum(~hotel_keep))

hotel_kept = hot_rev_df.loc[hotel_keep,:].copy()

print(sum(hotel_kept['reviewID'].isin(res_rev_df['reviewID'])))

# Change Y and YR to True and N and NR to False
res_rev_df.loc[res_rev_df['flagged']=='Y',  'flagged'] = True
res_rev_df.loc[res_rev_df['flagged']=='YR', 'flagged'] = True
res_rev_df.loc[res_rev_df['flagged']=='N',  'flagged'] = False
res_rev_df.loc[res_rev_df['flagged']=='NR',  'flagged'] = False

hotel_kept.loc[hotel_kept['flagged']=='Y',  'flagged'] = True
hotel_kept.loc[hotel_kept['flagged']=='YR', 'flagged'] = True
hotel_kept.loc[hotel_kept['flagged']=='N',  'flagged'] = False
hotel_kept.loc[hotel_kept['flagged']=='NR',  'flagged'] = False

combined = pd.concat([res_rev_df,hotel_kept]).reset_index(drop=True)

print(res_rev_df['flagged'].unique())
print(combined['flagged'].unique())

res_rev_df.to_csv(path+ '/clean_from_restaurants_reviews_UIC.csv')
hotel_kept.to_csv(path+ '/clean_from_hotel_reviews_UIC.csv')
combined.to_csv(path+ '/clean_reviews_UIC.csv')

['date', 'reviewID', 'reviewerID', 'reviewContent', 'rating', 'usefulCount', 'coolCount', 'funnyCount', 'flagged', 'businessID']
['date', 'reviewID', 'reviewerID', 'reviewContent', 'rating', 'usefulCount', 'coolCount', 'funnyCount', 'flagged', 'businessID']
181264
0
[False  True]
[False  True]


In [35]:
combined.head()

,date,reviewID,reviewerID,reviewContent,rating,usefulCount,coolCount,funnyCount,flagged,businessID
0,9/22/2012,GtwU21YOQn-wf4vWRUIx6w,bNYesZ944s6IJVowOnB0iA,"Unlike Next, which we'd eaten at the previous ...",5,0,0,0,False,pbEiXam9YJL3neCYHGwLUA
1,9/22/2012,0LpVTc3,TRKxLC3y-ZvP45e5iilMtw,Probably one of the best meals I've had ever. ...,5,0,0,0,False,pbEiXam9YJL3neCYHGwLUA
2,9/19/2012,tljtLzf68Fkwf,0EMm8umAqXZzyhxNpL4M9g,Service was impeccable. Experience and present...,3,2,0,0,False,pbEiXam9YJL3neCYHGwLUA
3,9/6/2012,iSN,DlwexC7z88ymAzu45skODw,"The problem with places like this, given the e...",3,8,0,3,False,pbEiXam9YJL3neCYHGwLUA
4,9/9/2012,Jmwrh7,kW2dk1CWihmh3g7k9N2G8A,I have no idea how to write my review - dining...,5,1,2,0,False,pbEiXam9YJL3neCYHGwLUA


In [36]:
res_rev_df['has_empty'] = res_rev_df.isna().sum(axis=1)>0

In [41]:
print(concat.columns)

Index(['coolCount', 'date', 'flagged', 'funnyCount', 'hotelID', 'rating',
       'restaurantID', 'reviewContent', 'reviewID', 'reviewerID',
       'usefulCount'],
      dtype='object')


In [42]:
# Percent flagged
total = res_rev_df.shape[0]

print("N #: ",res_rev_df.loc[res_rev_df.loc[:,'flagged']=='N',:].shape[0])
print("N %: ",res_rev_df.loc[res_rev_df.loc[:,'flagged']=='N',:].shape[0]/total)
print("")
print("Y #: ",res_rev_df.loc[res_rev_df.loc[:,'flagged']=='Y',:].shape[0])
print("Y %: ",res_rev_df.loc[res_rev_df.loc[:,'flagged']=='Y',:].shape[0]/total)
print("")
print("NR #: ",res_rev_df.loc[res_rev_df.loc[:,'flagged']=='NR',:].shape[0])
print("NR %: ",res_rev_df.loc[res_rev_df.loc[:,'flagged']=='NR',:].shape[0]/total)
print("")
print("YR #: ",res_rev_df.loc[res_rev_df.loc[:,'flagged']=='YR',:].shape[0])
print("YR %: ",res_rev_df.loc[res_rev_df.loc[:,'flagged']=='YR',:].shape[0]/total)

N #:  0
N %:  0.0

Y #:  0
Y %:  0.0

NR #:  0
NR %:  0.0

YR #:  0
YR %:  0.0
